In [1]:
import gensim
import sklearn
from sklearn.externals import joblib
from sklearn import svm

In [3]:
model = gensim.models.KeyedVectors.load_word2vec_format('vectors/GoogleNews-vectors-negative300.bin', binary=True)

In [172]:
def vectorize_word(word):
	if word in model.wv:
		return pd.Series(model.wv[word])
	else:
		# The pretrained word2vec model has dimensionality 300
		return pd.Series([np.nan] * 300)
def merge_vectors(v1, v2, method):
	print "method", method
	# Concat
	if method == 'concat':
		return pd.concat([v1, v2, y], axis = 1)
	elif method == 'diff':
		# Roller 2014 says to normalize the difference
		diff = v1 - v2
		return pd.concat([diff, y], axis = 1)
	elif method == 'asym':
		print "asym"
		# diff
		a = v1 - v2
		# squared diff - can't tell if they mean the mag^2 or ea element sq?
		b = pd.DataFrame(np.sqrt(np.square(a.values).sum(axis=1)))
		return pd.concat([a, b, y], axis = 1)
	elif method == 'cosine':
		#this fails???
		print "cosine"
		cos = pd.DataFrame(cosine_similarity(vectors_0.values, vectors_1.values).diagonal())
		return pd.concat([v1, v2, cos, y], axis = 1)

In [178]:
folder = 'turney2014'
method = 'concat'
for subset in ['test', 'train', 'val']:
    path_to_dataset = 'lexical_entailment/' + folder + '/data_lex_' + subset + '.tsv'
    path_to_vectorized = 'lexical_entailment/' + folder + '/data_lex_' + subset + '_vectorized.tsv'
    df = pd.read_csv(path_to_dataset, sep='\t', header=None)
    words_0 = df[0]
    words_1 = df[1]
    y = df[[2]]
    vectors_0 = words_0.apply(vectorize_word)
    vectors_1 = words_1.apply(vectorize_word)
    vectors_x = merge_vectors(vectors_0, vectors_1, method)
    vectors_x.to_csv(path_to_vectorized, sep='\t', header=False, index=False)

method concat
method concat
method concat


In [5]:
from sklearn.externals import joblib
import pandas as pd
from sklearn.metrics import accuracy_score

clf = joblib.load('models/svm.pkl') 

test_vectorized = pd.read_csv('lexical_entailment/bless2011/data_lex_test_vectorized.tsv', sep='\t', header=None)
val_vectorized = pd.read_csv('lexical_entailment/bless2011/data_lex_val_vectorized.tsv', sep='\t', header=None)

# Test and validation in one go bc i'm lazy
for test_name, test_df in zip(['test', 'val'], [test_vectorized, val_vectorized]):
	orig_rows, orig_cols = test_df.shape

	# Remove rows with NaN
	test_df.dropna(axis=0, inplace=True)

	# Count number of rows removed
	diff = orig_rows - test_df.shape[0]

	X = test_df.iloc[:, :-1]
	y = test_df.iloc[:, -1]

	preds = clf.predict(X)

	num_correct = accuracy_score(y, preds, normalize=False)

	print test_name, ": percentage non-nan correct:", num_correct/float(test_df.shape[0]) 
	print test_name, ": percentage correct overall", num_correct/float(orig_rows)
    

test : percentage non-nan correct: 0.923381396664
test : percentage correct overall 0.894794520548
val : percentage non-nan correct: 0.931343283582
val : percentage correct overall 0.871508379888


In [85]:
train = pd.read_csv('lexical_entailment/bless2011/data_lex_train_vectorized_asym.tsv', sep='\t', header=None)
train.dropna(axis=0, inplace=True)
X = train.iloc[:, :-1]
y = train.iloc[:, -1]

clf = svm.LinearSVC(class_weight='balanced')
clf.fit(X, y)

LinearSVC(C=1.0, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [99]:
test_vectorized = pd.read_csv('lexical_entailment/bless2011/data_lex_test_vectorized_asym.tsv', sep='\t', header=None)
test = pd.read_csv('lexical_entailment/bless2011/data_lex_test.tsv', sep='\t', header=None)
val_vectorized = pd.read_csv('lexical_entailment/bless2011/data_lex_val_vectorized_asym.tsv', sep='\t', header=None)

In [107]:
train_df = val_vectorized
test_df = test_vectorized
train_df.dropna(axis=0, inplace=True)
test_df.dropna(axis=0, inplace=True)
X = train_df.iloc[:, :-1]
y = train_df.iloc[:, -1]

In [101]:
preds = clf.predict(X)

In [102]:
print "precision", metrics.precision_score(y, preds)
print "recall", metrics.recall_score(y, preds)
print "f1", metrics.f1_score(y, preds)
print "True", metrics.accuracy_score(y[y == 1], preds[y == 1])
print "False", metrics.accuracy_score(y[y == 0], preds[y == 0])

precision 0.806451612903
recall 0.892857142857
f1 0.847457627119
True 0.892857142857
False 0.980456026059


In [103]:
p_hat = clf.coef_[0][:301]

In [104]:
def normalize(v):
  '''normalize' a vector, in the traditional linear algebra sense.'''
  norm=np.linalg.norm(v)
  if norm==0:
    return v
  return v/norm
def reject(A):

  '''Create a 'projection', and subract it from the original vector'''
  B = p_hat
  project = np.linalg.linalg.dot(A, normalize(B)) * normalize(B)
  return A - project


In [105]:
X_hat = X.apply(reject, axis=1)

In [108]:
clf_hat = svm.LinearSVC()
clf_hat.fit(X_hat, y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [109]:
X_test = test_df.iloc[:, :-1]
y_test = test_df.iloc[:, -1]
preds_hat = clf_hat.predict(X_test)
print "precision", metrics.precision_score(y_test, preds_hat)
print "recall", metrics.recall_score(y_test, preds_hat)
print "f1", metrics.f1_score(y_test, preds_hat)
print "True", metrics.accuracy_score(y_test[y_test == 1], preds_hat[y_test == 1])
print "False", metrics.accuracy_score(y_test[y_test == 0], preds_hat[y_test == 0])

precision 0.416243654822
recall 0.30258302583
f1 0.350427350427
True 0.30258302583
False 0.964788732394


In [110]:
print "precision", metrics.precision_score(y, preds)
print "recall", metrics.recall_score(y, preds)
print "f1", metrics.f1_score(y, preds)
print "True", metrics.accuracy_score(y[y == 1], preds[y == 1])
print "False", metrics.accuracy_score(y[y == 0], preds[y == 0])

precision 0.806451612903
recall 0.892857142857
f1 0.847457627119
True 0.892857142857
False 0.980456026059


In [157]:
clf = joblib.load('models/svm.pkl') 

In [170]:
fp = 'lexical_entailment/bless2011/data_lex_test.tsv'
test = pd.read_csv(fp, sep='\t', header=None)
test_vectorized = pd.read_csv('lexical_entailment/bless2011/data_lex_test_vectorized.tsv', sep='\t', header=None)

In [171]:

test_df = test_vectorized
test_df.dropna(axis=0, inplace=True)

X = test_df.iloc[:, :-1]
y = test_df.iloc[:, -1]

preds = clf.predict(X)
true_count = y.sum()
preds_count = preds.sum()
print "precision", metrics.precision_score(y, preds)
print "recall", metrics.recall_score(y, preds)
print "f1", metrics.f1_score(y, preds)
print "True", metrics.accuracy_score(y[y == 1], preds[y == 1])
print "False", metrics.accuracy_score(y[y == 0], preds[y == 0])
print true_count, preds_count, orig_rows

num_correct = metrics.accuracy_score(y, preds, normalize=False)

print test_name, ": percentage non-nan correct:", num_correct/float(test_df.shape[0])
print test_name, ": percentage correct overall", num_correct/float(orig_rows)

precision 0.406113537118
recall 0.343173431734
f1 0.372
True 0.343173431734
False 0.958358848745
271 229 358
val : percentage non-nan correct: 0.911224201301
val : percentage correct overall 9.00279329609


In [140]:
#Vectorize train/test/val sets

path_to_dataset = 'lexical_entailment/turney2014/data_lex_val.tsv'
path_to_vectorized = 'lexical_entailment/turney2014/data_lext_val_vectorized.tsv'
df = pd.read_csv(path_to_dataset, sep='\t', header=None)

words_0 = df[0]
words_1 = df[1]
y = df[[2]]
method = 'concat'
def vectorize_word(word):
	if word in model.wv:
		return pd.Series(model.wv[word])
	else:
		# The pretrained word2vec model has dimensionality 300
		return pd.Series([0] * 300)

vectors_0 = words_0.apply(vectorize_word)
vectors_1 = words_1.apply(vectorize_word)
def merge_vectors(v1, v2, method):
	print "method", method
	# Concat
	if method == 'concat':
		return pd.concat([vectors_0, vectors_1, y], axis = 1)
	elif method == 'diff':
		# Roller 2014 says to normalize the difference
		diff = vectors_0 - vectors_1
		return pd.concat([diff, y], axis = 1)
	elif method == 'asym':
		print "asym"
		# diff
		a = vectors_0 - vectors_1
		# squared diff - can't tell if they mean the mag^2 or ea element sq?
		b = pd.DataFrame(np.sqrt(np.square(a.values).sum(axis=1)))
		return pd.concat([a, b, y], axis = 1)
	elif method == 'cosine':
		#this fails???
		print "cosine"
		cos = pd.DataFrame(cosine_similarity(vectors_0.values, vectors_1.values).diagonal())
		return pd.concat([v1, v2, cos, y], axis = 1)

vectors_x = merge_vectors(vectors_0, vectors_1, method)
vectors_x.to_csv(path_to_vectorized, sep='\t', header=False, index=False)

method concat


In [151]:
X

,0,1
0,absentia,sentence
1,accounting,management
2,activist,federation
3,advisories,federation
4,aerospace,management
5,agency,government
6,agriculture,environment
7,agriculture,management
8,ambassador,chairman
9,amendment,bill
